# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 5 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'proficient page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'API endpoints', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'Join page', 'url': 'https://huggingface.co/join'},
  {'type': 'Homepage', 'url': 'https://huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-4.7-Flash
Updated
6 days ago
•
450k
•
1.2k
nvidia/personaplex-7b-v1
Updated
3 days ago
•
34.9k
•
1.02k
microsoft/VibeVoice-ASR
Updated
5 days ago
•
46.9k
•
536
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
3 days ago
•
91.8k
•
495
unsloth/GLM-4.7-Flash-GGUF
Updated
2 days ago
•
228k
•
332
Browse 2M+ models
Spaces
Running
on
Zero
Featured
700
Qwen3-TTS Demo
🎙
700
Convert text to speech with custom voices and cloning
Running
on
Zero
Featured
1.16k
Qwen Image Multiple Angles 3D Camera
🎥
1.16k
Adjust camera angles in images us

In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-4.7-Flash\nUpdated\n6 days ago\n•\n450k\n•\n1.2k\nnvidia/personaplex-7b-v1\nUpdated\n3 days ago\n•\n34.9k\n•\n1.02k\nmicrosoft/VibeVoice-ASR\nUpdated\n5 days ago\n•\n46.9k\n•\n536\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n3 days ago\n•\n91.8k\n•\n495\nunsloth/GLM-4.7-Flash-GGUF\nUpdated\n2 days ago\n•\n228k\n•\n332\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n7

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant AI community and collaboration platform building the future of machine learning. It serves as a central hub where machine learning engineers, scientists, and end users come together to create, share, explore, and experiment with open-source models, datasets, and applications.

With a fast-growing, global community and an extensive ecosystem, Hugging Face empowers the next generation of AI practitioners by enabling seamless collaboration and knowledge sharing. The company is committed to fostering an **open and ethical AI future** and stands at the heart of the AI revolution with some of the most popular open-source ML tools and a cutting-edge research science team.

---

## What Hugging Face Offers

- **Hugging Face Hub:**  
  A central platform hosting over **2 million machine learning models**, datasets, and apps in various modalities including text, image, video, audio, and even 3D data. It enables unlimited public projects, discovery, and collaboration.

- **Spaces:**  
  Interactive machine learning applications powered by the community and Hugging Face infrastructure. Spaces enable users to demo, run, and share exciting AI apps, such as text-to-speech conversion, 3D image manipulation, and AI-generated art tools.

- **Open Source ML Stack:**  
  Reliable infrastructure and tools that help move ML development faster, supporting engineering productivity at scale.

- **Datasets:**  
  Access to over **500,000 curated datasets** for diverse machine learning tasks, updated frequently by community and industry contributors.

- **Community:**  
  A globally distributed, active community collaborating on state-of-the-art ML solutions, sharing results, best practices, and innovations.

- **Enterprise Solutions:**  
  Tailored offerings for companies looking to harness and deploy AI models securely and efficiently at scale.

---

## Company Culture

- **Collaboration & Openness:**  
  Hugging Face thrives on openness and sharing. The platform is designed to help people **learn from one another and build trust through collaboration** on open-source AI projects.

- **Innovation at the Edge:**  
  With a scientific team pushing technological boundaries, Hugging Face fosters an environment where creativity and technical excellence flourish.

- **Community-Driven:**  
  Supporting the AI ecosystem and its future leaders is core. The company encourages community contributions and supports ethical AI development.

- **Diversity of Modalities:**  
  Hugging Face embraces innovation beyond traditional text modeling, exploring audio, video, and 3D applications, demonstrating its culture of broad exploration and inclusion across AI disciplines.

---

## Who Uses Hugging Face?

- **Machine Learning Practitioners:**  
  From beginners to experts, ML engineers and researchers use the platform to host, find, and contribute models and datasets.

- **Developers & Data Scientists:**  
  Building AI-powered applications and data pipelines leveraging the immense repository of resources.

- **Enterprises:**  
  Organizations requiring scalable, customizable AI infrastructure and management solutions rely on Hugging Face’s enterprise offerings.

- **Educators & Students:**  
  Using the platform as an educational resource to learn, experiment, and build machine learning portfolios.

---

## Careers at Hugging Face

Hugging Face is continuously looking for talented individuals passionate about AI and open-source development.

- **Open Roles:** Engineering, Research Science, Product Management, Community, and more.
- **Work Environment:** Dynamic, collaborative, innovative, and supportive of continuous learning and growth.
- **Mission-Driven:** Join a company dedicated to building AI for the future in a responsible and community-first way.

For current job openings and application details, visit Hugging Face’s careers page on their website.

---

## Connect & Discover

- Access the platform: [huggingface.co](https://huggingface.co)  
- Join the community via Discord, GitHub, Twitter, LinkedIn  
- Explore over **2 million models**, **500k+ datasets**, and **1 million+ AI-powered applications**

---

**Hugging Face** — Empowering the AI community to build the future, together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future. It is a collaboration platform at the heart of the machine learning revolution, empowering engineers, scientists, and AI practitioners to create, share, and innovate together. Leveraging a fast-growing community and a rich ecosystem of open-source tools, Hugging Face accelerates advances in artificial intelligence across multiple modalities like text, images, video, audio, and even 3D data.

---

## What We Offer

- **Models:** Access and contribute to over 2 million machine learning models spanning various AI domains — from natural language processing to computer vision and speech recognition.
- **Datasets:** Browse and host over 500,000 open datasets, facilitating research and development with diverse data sources.
- **Spaces:** Build and share machine learning applications using “Spaces,” where dynamic demos and tools can run, powered by zero-cost compute platforms.
- **Community:** Join a vibrant community of over 79,000 AI enthusiasts and professionals collaborating, sharing papers, code, and learning resources.
- **Enterprise:** Tailored AI solutions and support for businesses looking to leverage advanced machine learning capabilities.
- **Learning & Docs:** Extensive documentation, educational blog articles, and forums for continuous learning and community support.

---

## Company Culture

Hugging Face fosters an open, ethical, and collaborative AI ecosystem. The company believes in democratizing good machine learning, embracing transparency, sharing, and collective knowledge growth—one commit at a time.

- **Open Source Focus:** Hugging Face champions open-source contributions and shared innovation.
- **Community-Driven:** The platform is built for and by the AI community, encouraging active participation and knowledge exchange.
- **Innovation at the Edge:** The talented science team constantly explores cutting-edge technologies to push AI boundaries.
- **Diversity & Inclusivity:** Committed to building a welcoming environment where diverse perspectives drive better AI solutions.

---

## Customers & Partners

Hugging Face serves a wide spectrum of customers:

- **Individual Researchers & Developers:** Leveraging the platform to build portfolios, share models, and collaborate.
- **Startups & Enterprises:** Applying Hugging Face’s tools and infrastructure for scalable AI deployments and custom applications.
- **Academic Institutions:** Using datasets and models to accelerate research in AI and machine learning.
- **AI Organizations & Tech Giants:** Engaging with Hugging Face for cutting-edge open source tools and research collaborations.

---

## Careers & Opportunities

Join Hugging Face to be part of an exciting AI journey with meaningful work and impactful innovation.

- **Open Positions:** The company frequently hires across engineering, research, product, and community roles.
- **Growth & Learning:** Employees have access to continuous learning tracks such as “Hugging Face Fundamentals” on platforms like DataCamp.
- **Collaborative Environment:** Work alongside a diverse and passionate team with a shared mission to democratize machine learning.
- **Join Us:** As stated by Hugging Face, if you are passionate about AI and believe in building an ethical, open AI future, "why don't you join us!"

---

## Connect with Hugging Face

- Website: [https://huggingface.co](https://huggingface.co)
- Community & Socials: GitHub, Twitter, LinkedIn, Discord
- Press Contact: Reach out via their official website for press and media inquiries.

---

**Colors & Brand Identity:**

- Primary Colors: #FFD21E (Yellow), #FF9D00 (Orange), #6B7280 (Gray)
- Logos and brand assets available for use in .svg, .png, and .ai formats.

---

Hugging Face is your gateway to the AI community that’s shaping the future—come build the next generation of machine learning together!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>